# Assignment 10: A* pathfinding
A* pathfinding is a commonly used algorithm for find the shortest traversable distance in applications like route finding in maps to AI pathfinding in video games. It is similar to Dijkstra's algorithm, but rather than traversing all nodes, it uses a cost calculation that allows the untraversed nodes to be sorted in a heap structure. For example, let's say we have a binary array of size (5, 5), where each value is a map tile:
```
[0  0  0  0  0
 0  0  1  0* 0
 0  0  1  0  0
 0  0  0  1  0
 0  0  0  0  0]
```
Your goal is to traverse the map from the bottom left point to the tile denoted by the asterisk. Any space on the map occupied by 0 is considered walkable, whereas 1 represents an impassable barrier. In this exercise we will create an algorithm that efficiently calculates the shortest walkable path to the goal.

## Define nodes
Create a class `Node` for each tile in the map. These will contain:
1. A value `g`, the distance from the start node to this node
2. A value `h`, the heuristic, or estimated distance to the end node
3. A value `f`, the total cost of the node as defined by `f + g`
4. The map coordinate `coord`
5. A boolean designation `walkable`
6. A list of the the neighboring nodes `neighbors`
7. A function for getting the `neighbors`

In [1]:
import math

class Node:
    def __init__(self, coord, walkable=True):
        self.coord = coord
        self.g = 0
        self.h = 0
        self.f = 0
        self.walkable = walkable
        self.neighbors = []
        self.parent = None
        
    def calculate_h(self, end_node):
        self.h = math.sqrt((self.coord[0] - end_node.coord[0])**2 + (self.coord[1] - end_node.coord[1])**2)

    def calculate_f(self):
        self.f = self.g + self.h

    def add_neighbor(self, neighbor):
        self.neighbors.append(neighbor)
        
    def get_neighbors(self):
        return self.neighbors        

## The algorithm
The main function will need to accept three arguments: the map array, the starting position, and the end position. Generally, you will have two lists of nodes: one for nearby candidate nodes that have yet to be traversed (`open_list`), and one of nodes you've already visited (`closed_list`). Here are the steps:
1. Add the starting node to the open list.
2. Repeat the following:
    A) Look for the lowest F cost square on the open list. We refer to this as the current square.
    B). Switch it to the closed list.
    C) For each of the 8 squares adjacent to this current square …
        - If it is not walkable or if it is on the closed list, ignore it. Otherwise do the following.
        - If it isn’t on the open list, add it to the open list.
        - Make the current square the parent of this square.
        - Record the F, G, and H costs of the square.
        -If it is on the open list already, check to see if this path to that square is better, using G cost as the measure. A lower G cost means that this is a better path. If so, change the parent of the square to the current square, and recalculate the G and F scores of the square. If you are keeping your open list sorted by F score, you may need to resort the list to account for the change.
    D) Stop when you:
        - Add the target square to the closed list, in which case the path has been found, or
        - Fail to find the target square, and the open list is empty. In this case, there is no path.
3. Save the path. Working backwards from the target square, go from each square to its parent square until you reach the starting square. That is your path.

In this case, the `open_list` will be a minimum heap. Create a class `Heap` the has:
1. `heap`: a list of nodes
2. A function `append` for adding nodes to the heap.
3. A function `sift_up` for sifting a node to the top of the heap
4. A function `sort` for sorting the heap such that the minimum is on top.

In [2]:
class Heap:
    def __init__(self):
        self.heap = []

    def append(self, node):
        self.heap.append(node)
        self.sift_up(len(self.heap) - 1)

    def sift_up(self, index):
        parent_index = (index - 1) // 2
        while index > 0 and self.heap[index].f > self.heap[parent_index].f:
            self.heap[index], self.heap[parent_index] = self.heap[parent_index], self.heap[index]
            index = parent_index

    def sort(self):
        self.heap.sort(key=lambda x: x.f)
        
    def _swap(self,i,j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]

    def pop(self):
        if self.is_empty():
            raise IndexError("Heap is empty.")
        min_node = self.heap[0]
        self.heap[0] = self.heap[-1]
        del self.heap[-1]
        self.sift_down(0)
        return min_node
            

    def sift_down(self, index):
        while index * 2 + 1 < len(self.heap):
            left_child_index = index * 2 + 1
            right_child_index = index * 2 + 2
            min_index = left_child_index
            if right_child_index < len(self.heap) and self.heap[right_child_index].f < self.heap[min_index].f:
                min_index = right_child_index
            if self.heap[min_index].f < self.heap[index].f:
                self._swap(index, min_index)
                index = min_index
            else:
                break
                
    def is_empty(self):
        return len(self.heap) == 0

Now implement a main function that performs the tasks of the algorithm

In [3]:
def main(map_array, start_pos, end_pos):
    open_list = Heap()
    closed_list = []

    start_node = Node(start_pos)
    end_node = Node(end_pos)
    open_list.append(start_node)

    while open_list.heap:
        current_node = open_list.heap[0]
        closed_list.append(current_node)
        del open_list.heap[0]

        if current_node.coord == end_pos:
            path = []
            while current_node:
                path.append(current_node.coord)
                current_node = current_node.parent
            return path[::-1]

        for neighbor_coord in get_neighbors(map_array, current_node.coord):
            if neighbor_coord in [node.coord for node in closed_list]:
                continue

            neighbor_node = Node(neighbor_coord)
            neighbor_node.parent = current_node

            if neighbor_coord not in [node.coord for node in open_list.heap]:
                neighbor_node.g = current_node.g + 1
                neighbor_node.calculate_h(end_node)
                neighbor_node.calculate_f()
                open_list.append(neighbor_node)
                open_list.sort()
            else:
                for node in open_list.heap:
                    if node.coord == neighbor_coord:
                        temp_g = current_node.g + 1
                        if temp_g < node.g:
                            node.g = temp_g
                            node.parent = current_node
                            node.calculate_f()
                            open_list.sort()

    return None

def get_neighbors(map_array, coord):
    neighbors = []
    rows = len(map_array)
    cols = len(map_array[0])
    for dr in [-1, 0, 1]:
        for dc in [-1, 0, 1]:
            if dr == 0 and dc == 0:
                continue
            new_row = coord[0] + dr
            new_col = coord[1] + dc
            if 0 <= new_row < rows and 0 <= new_col < cols and map_array[new_row][new_col]:
                neighbors.append((new_row, new_col))
    return neighbors

In [4]:
def findPath(map_array, start_pos, end_pos):
    path = main(map_array, start_pos, end_pos)
    if path:
        print("Path found:", path)
    else:
        print("No path found.")
    

In [5]:
map_array5 = [
    [1, 1, 1, 1, 1],
    [1, 0, 1, 0, 1],
    [1, 1, 1, 1, 1],
    [1, 0, 1, 0, 1],
    [1, 1, 1, 1, 1]
]

start_pos5 = (0, 0)
end_pos5 = (4, 4)


findPath(map_array5, start_pos5, end_pos5)

Path found: [(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 4)]


In [6]:
map_array10 = [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 1, 1, 1, 0, 0, 1],
    [1, 1, 1, 0, 1, 1, 0, 0, 1, 1],
    [1, 1, 1, 0, 0, 0, 0, 0, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 1, 1, 1, 1, 0, 0, 1],
    [1, 1, 0, 1, 1, 1, 0, 1, 1, 1],
    [1, 1, 0, 0, 0, 0, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]

start_pos10 = (0, 0)
end_pos10 = (9, 9)
findPath(map_array10, start_pos10, end_pos10)

Path found: [(0, 0), (1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 9)]


In [7]:
map_array20 = [
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1],
    [0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1],
    [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1],
    [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]

start_pos20 = (0, 0)
end_pos20 = (19, 19)
findPath(map_array20, start_pos20, end_pos20)

Path found: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 4), (2, 4), (3, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (5, 12), (6, 13), (7, 13), (8, 13), (9, 13), (10, 13), (11, 13), (12, 13), (13, 13), (14, 13), (15, 13), (16, 13), (17, 13), (18, 13), (19, 14), (19, 15), (19, 16), (19, 17), (19, 18), (19, 19)]
